# SwiFT Pipeline with ADNI Degradation Data

This notebook demonstrates the complete SwiFT (Swin 4D fMRI Transformer) pipeline applied to ADNI (Alzheimer's Disease Neuroimaging Initiative) data for cognitive decline prediction.

## Overview

The notebook covers:
1. **Setup and Configuration** - Import libraries and configure paths
2. **Stage 1: Contrastive Pretraining** - Self-supervised learning on unlabeled fMRI data
3. **Stage 2: Supervised Fine-tuning** - Training on degradation prediction tasks
4. **Evaluation** - Test set evaluation and metrics

## SwiFT Methodology

SwiFT uses a two-stage approach:
- **Pretraining**: Learn robust fMRI representations using contrastive learning (no labels needed)
- **Fine-tuning**: Adapt the pretrained model to specific downstream tasks (degradation prediction)

This approach allows efficient use of limited labeled data by leveraging unlabeled data during pretraining.

## 1. Setup and Imports

In [18]:
torch.save(x, '/sci/nosnap/arieljaffe/sagi.nathan/shared_fmri_data/all_4d_downsampled_35x35x35.pt')

In [2]:
import sys
import os



import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# SwiFT pipeline imports
from data.prepare_adni_data import prepare_adni_datasets, create_adni_dataloaders
from models.swin4d_transformer_ver7 import SwinTransformer4D
from models.heads import ContrastiveHead, ClassificationHead
from training.losses import NTXentLoss
from configs.config_pretrain import MODEL_CONFIG, CONTRASTIVE_CONFIG, TRAIN_CONFIG, DATA_CONFIG
from configs.config_finetune import TASK_CONFIG, HEAD_CONFIG, TRAIN_CONFIG as FINETUNE_TRAIN_CONFIG

print("=" * 80)
print("SWIFT PIPELINE WITH ADNI DEGRADATION DATA")
print("=" * 80)
print("\n✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

SWIFT PIPELINE WITH ADNI DEGRADATION DATA

✓ All imports successful!
PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA device: NVIDIA L4


## 2. Configuration

Set up paths to your ADNI data files and training parameters.

# Paths to your ADNI data
# Updated to use shared_fmri_data with new resolution: 35x37x35x10
USE_DUMMY_DATA = False  # Using real data from shared_fmri_data

if USE_DUMMY_DATA:
    BASE_DATA_PATH = "../data_dummy"
    DATA_PATH = f"{BASE_DATA_PATH}/all_4d_downsampled.pt"
    LABELS_PATH = f"{BASE_DATA_PATH}/imageID_to_labels.json"
    INFO_PATH = f"{BASE_DATA_PATH}/index_to_name.json"
    print("⚠️  Using DUMMY data for testing")
    print("   Run 'create_dummy_adni_data.py' first to generate dummy data")
else:
    # Using real ADNI data from shared_fmri_data
    BASE_DATA_PATH = "/sci/nosnap/arieljaffe/sagi.nathan"
    DATA_PATH = f"{BASE_DATA_PATH}/shared_fmri_data/all_4d_downsampled.pt"
    LABELS_PATH = f"{BASE_DATA_PATH}/imageID_to_labels.json"
    INFO_PATH = f"{BASE_DATA_PATH}/index_to_name.json"
    print("✓ Using REAL ADNI data from shared_fmri_data")

print(f"Data path: {DATA_PATH}")

# Task configuration - choose which degradation task to predict
DEGRADATION_TASKS = [
    "degradation_binary_1year",
    # "degradation_binary_2years",  # Uncomment for multi-task learning
    # "degradation_binary_3years",
]

# Spatial configuration - Updated for new resolution
# Using Resize3D with scale_factor=0.7 to achieve ~35x37x35 spatial resolution
TARGET_SPATIAL_SIZE = (35, 37, 35)  # Model expects 35x37x35 input (changed from 96x96x96)
SCALE_FACTOR = 0.7  # Resize3D scale factor to achieve target spatial size
WINDOW_SIZE = 10  # Temporal window size (changed from 20)
STRIDE = 5  # Temporal stride (changed from 10)

# Training configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAIN_EPOCHS = 5  # Increase for real training (e.g., 100-300)
FINETUNE_EPOCHS = 10  # Increase for real training (e.g., 50-100)

print(f"\nDevice: {DEVICE}")
print(f"Target spatial size: {TARGET_SPATIAL_SIZE}")
print(f"Scale factor: {SCALE_FACTOR}")
print(f"Temporal window size: {WINDOW_SIZE}")
print(f"Temporal stride: {STRIDE}")
print(f"Task(s): {DEGRADATION_TASKS}")
print(f"Pretraining epochs: {PRETRAIN_EPOCHS}")
print(f"Fine-tuning epochs: {FINETUNE_EPOCHS}")


In [4]:
# Paths to your ADNI data
# Updated to use disk-based preprocessed windows
USE_DUMMY_DATA = False  # Using real preprocessed data

if USE_DUMMY_DATA:
    BASE_DATA_PATH = "../data_dummy"
    PREPROCESSED_DIR = f"{BASE_DATA_PATH}/preprocessed_windows"
    LABELS_PATH = f"{BASE_DATA_PATH}/imageID_to_labels.json"
    INFO_PATH = f"{BASE_DATA_PATH}/index_to_name.json"
    print("⚠️  Using DUMMY data for testing")
    print("   Run 'create_dummy_adni_data.py' first to generate dummy data")
else:
    # Using real preprocessed ADNI data
    BASE_DATA_PATH = "/sci/nosnap/arieljaffe/sagi.nathan/shared_fmri_data"
    PREPROCESSED_DIR = f"{BASE_DATA_PATH}/preprocessed_windows"
    LABELS_PATH = f"{PREPROCESSED_DIR}/imageID_to_labels.json"
    INFO_PATH = f"{PREPROCESSED_DIR}/index_to_info.json"
    print("✓ Using REAL preprocessed ADNI data")

print(f"Preprocessed directory: {PREPROCESSED_DIR}")

# Task configuration - choose which degradation task to predict
DEGRADATION_TASKS = [
    "degradation_binary_1year",
    # "degradation_binary_2years",  # Uncomment for multi-task learning
    # "degradation_binary_3years",
]

# Training configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAIN_EPOCHS = 5  # Increase for real training (e.g., 100-300)
FINETUNE_EPOCHS = 10  # Increase for real training (e.g., 50-100)

print(f"\nDevice: {DEVICE}")
print(f"Task(s): {DEGRADATION_TASKS}")
print(f"Pretraining epochs: {PRETRAIN_EPOCHS}")
print(f"Fine-tuning epochs: {FINETUNE_EPOCHS}")

✓ Using REAL preprocessed ADNI data
Preprocessed directory: /sci/nosnap/arieljaffe/sagi.nathan/shared_fmri_data/preprocessed_windows

Device: cuda
Task(s): ['degradation_binary_1year']
Pretraining epochs: 5
Fine-tuning epochs: 10


## 3. Stage 1: Contrastive Pretraining (Self-Supervised)

In this stage, we train the SwiFT encoder using contrastive learning **without labels**. The model learns to create meaningful representations of fMRI data by:
- Taking two different temporal windows from the same scan
- Learning to make their representations similar (positive pairs)
- While making representations from different scans dissimilar (negative pairs)

This is self-supervised learning - no degradation labels are needed!

### 3.1 Prepare Pretraining Datasets

In [5]:
print("=" * 80)
print("STAGE 1: CONTRASTIVE PRETRAINING")
print("=" * 80)

# Prepare datasets for pretraining using disk-based loading
from data.dataset_adni_disk import prepare_adni_disk_datasets

pretrain_datasets = prepare_adni_disk_datasets(
    preprocessed_dir=PREPROCESSED_DIR,
    task_names=DEGRADATION_TASKS,  # Not used in pretraining
    stage="pretrain",
    val_split=0.1,
    test_split=0.2,
    seed=42,
    target_spatial_size=None,  # Already preprocessed
    handle_nan='skip',
)

# Create dataloaders
from torch.utils.data import DataLoader

train_loader = DataLoader(
    pretrain_datasets['train_dataset'],
    batch_size=TRAIN_CONFIG["batch_size"],
    shuffle=True,
    num_workers=4,  # Parallel loading
)

val_loader = DataLoader(
    pretrain_datasets['val_dataset'],
    batch_size=TRAIN_CONFIG["batch_size"],
    shuffle=False,
    num_workers=4,
)

print(f"\n✓ Dataloaders created:")
print(f"  - Train batches: {len(train_loader)}")
print(f"  - Val batches: {len(val_loader)}")

STAGE 1: CONTRASTIVE PRETRAINING
PREPARING DISK-BASED ADNI DATASETS FOR SWIFT PIPELINE

Loading window index from /sci/nosnap/arieljaffe/sagi.nathan/shared_fmri_data/preprocessed_windows/window_index.json...
  Total windows: 18792

Loading labels from /sci/nosnap/arieljaffe/sagi.nathan/shared_fmri_data/preprocessed_windows/imageID_to_labels.json...
  Labels for 829 scans

Data split by subjects (no data leakage):
  Train: 488 subjects, 13176 samples
  Val:   69 subjects, 1863 samples
  Test:  139 subjects, 3753 samples

CREATING CONTRASTIVE PRETRAINING DATASETS
  Contrastive dataset: 13176 valid samples from 488 scans
  Contrastive dataset: 1863 valid samples from 69 scans

DATA PREPARATION COMPLETE


✓ Dataloaders created:
  - Train batches: 3294
  - Val batches: 466


In [ ]:
train_loader.dataset[0]

### 3.2 Initialize Models for Pretraining

In [8]:
print("Initializing models for pretraining...")
print("\n" + "=" * 60)
print("SwiFT Configuration (adapted for ADNI data)")
print("=" * 60)
print(f"  img_size:           {MODEL_CONFIG['img_size']}")
print(f"  patch_size:         {MODEL_CONFIG['patch_size']}")
print(f"  window_size:        {MODEL_CONFIG['window_size']}")
print(f"  first_window_size:  {MODEL_CONFIG['first_window_size']}")
print(f"  embed_dim:          {MODEL_CONFIG['embed_dim']}")
print(f"  depths:             {MODEL_CONFIG['depths']}")
print(f"  num_heads:          {MODEL_CONFIG['num_heads']}")
print(f"  c_multiplier:       {MODEL_CONFIG['c_multiplier']}")
print(f"  last_layer_full_MSA:{MODEL_CONFIG['last_layer_full_MSA']}")
print("=" * 60)
print(f"  batch_size:         {TRAIN_CONFIG['batch_size']}")
print(f"  learning_rate:      {TRAIN_CONFIG['learning_rate']}")
print(f"  temperature:        {TRAIN_CONFIG['temperature']}")
print("=" * 60 + "\n")

# SwiFT Encoder (4D Swin Transformer)
encoder = SwinTransformer4D(**MODEL_CONFIG).to(DEVICE)

# Contrastive projection head
contrastive_head = ContrastiveHead(**CONTRASTIVE_CONFIG).to(DEVICE)

# Print model info
total_params = sum(p.numel() for p in encoder.parameters())
head_params = sum(p.numel() for p in contrastive_head.parameters())
print(f"\n✓ Models initialized on {DEVICE}")
print(f"  - Encoder parameters: {total_params:,}")
print(f"  - Contrastive head parameters: {head_params:,}")

# Setup training with SwiFT defaults
criterion = NTXentLoss(
    device=DEVICE,
    batch_size=TRAIN_CONFIG["batch_size"],  # SwiFT default: 8
    temperature=TRAIN_CONFIG["temperature"],  # 0.1
)

optimizer = optim.AdamW(
    list(encoder.parameters()) + list(contrastive_head.parameters()),
    lr=TRAIN_CONFIG["learning_rate"],  # SwiFT default: 5e-5
)

print(f"✓ Loss function: NTXentLoss (temperature={TRAIN_CONFIG['temperature']})")
print(f"✓ Optimizer: AdamW (lr={TRAIN_CONFIG['learning_rate']}, batch_size={TRAIN_CONFIG['batch_size']})")

Initializing models for pretraining...
img_size:  (35, 37, 35, 10)
patch_size:  (5, 5, 5, 1)
patch_dim:  (7, 7, 7, 10)

✓ Models initialized on cuda
  - Encoder parameters: 4,125,132
  - Contrastive head parameters: 120,800
✓ Loss function: NTXentLoss (temperature=0.5)
✓ Optimizer: AdamW (lr=5e-05)


### 3.3 Run Contrastive Pretraining

In [ ]:
from datetime import datetime

# Create timestamped checkpoint directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = f"../checkpoints/pretrain_run_{timestamp}"
os.makedirs(run_dir, exist_ok=True)

print(f"\nCreated run directory: {run_dir}")
print(f"Starting pretraining ({PRETRAIN_EPOCHS} epochs)...")
print("Note: Increase epochs for real training (e.g., 100-300)\n")

encoder.train()
contrastive_head.train()

# Calculate checkpoint intervals (at 25%, 50%, 75%, 100%)
checkpoint_intervals = [0.25, 0.50, 0.75, 1.0]
total_steps = PRETRAIN_EPOCHS
checkpoint_steps = [int(total_steps * interval) for interval in checkpoint_intervals]

for epoch in range(PRETRAIN_EPOCHS):
    epoch_loss = 0.0
    num_batches = 0
    
    for batch_idx, (view1, view2) in enumerate(train_loader):
        view1, view2 = view1.to(DEVICE), view2.to(DEVICE)
        
        # Forward pass
        optimizer.zero_grad()
        
        # Encode both views
        features1 = encoder(view1)
        features2 = encoder(view2)
        
        # Project to embedding space
        embeddings1 = contrastive_head(features1)
        embeddings2 = contrastive_head(features2)
        
        # Compute contrastive loss
        loss = criterion(embeddings1, embeddings2)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1
        
        # Limit batches for demo (remove for real training)
        if num_batches >= 10:
            break
    
    avg_loss = epoch_loss / num_batches
    print(f"  Epoch {epoch+1}/{PRETRAIN_EPOCHS}: Loss = {avg_loss:.4f}")
    
    # Save checkpoints at 25%, 50%, 75%, and 100%
    current_epoch = epoch + 1
    if current_epoch in checkpoint_steps:
        checkpoint_idx = checkpoint_steps.index(current_epoch)
        checkpoint_name = f"checkpoint_{(checkpoint_idx + 1) * 25}percent_epoch{current_epoch}.pth"
        checkpoint_path = os.path.join(run_dir, checkpoint_name)
        
        torch.save({
            'model_state_dict': encoder.state_dict(),
            'head_state_dict': contrastive_head.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': current_epoch,
            'loss': avg_loss,
            'config': MODEL_CONFIG,
        }, checkpoint_path)
        
        print(f"    ✓ Saved checkpoint: {checkpoint_name}")

print("\n✓ Pretraining complete!")
print("  The encoder has learned to create meaningful fMRI representations")
print(f"  All checkpoints saved to: {run_dir}")


Starting pretraining (5 epochs)...
Note: Increase epochs for real training (e.g., 100-300)

  Epoch 1/5: Loss = 1.9540
  Epoch 1/5: Loss = 1.9540
  Epoch 2/5: Loss = 1.9362
  Epoch 2/5: Loss = 1.9362


KeyboardInterrupt: 

### 3.4 Save Pretrained Model

In [ ]:
pretrained_path = os.path.join(run_dir, "pretrained_adni_final.pth")

torch.save({
    'model_state_dict': encoder.state_dict(),
    'head_state_dict': contrastive_head.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': MODEL_CONFIG,
}, pretrained_path)

print(f"✓ Saved pretrained encoder to: {pretrained_path}")
print(f"  Run directory: {run_dir}")
print("  Note: Only the encoder is saved, not the contrastive head")

✓ Saved pretrained encoder to: ../checkpoints/pretrained_adni.pth
  Note: Only the encoder is saved, not the contrastive head


## 4. Stage 2: Supervised Fine-tuning for Degradation Prediction

Now we use the pretrained encoder for the downstream task: predicting cognitive decline (degradation).

Key differences from pretraining:
- **Uses labels**: We now have degradation labels (0 or 1) from `imageID_to_labels.json`
- **Frozen encoder**: The pretrained encoder is typically frozen (not trained)
- **Classification head**: We train only a small classification head on top
- **Fewer epochs needed**: Transfer learning converges faster than training from scratch

### 4.1 Prepare Fine-tuning Datasets with Labels

In [ ]:
print("=" * 80)
print("STAGE 2: SUPERVISED FINE-TUNING")
print("=" * 80)

# Prepare datasets for finetuning with labels
finetune_datasets = prepare_adni_datasets(
    data_path=DATA_PATH,
    labels_path=LABELS_PATH,
    info_path=INFO_PATH,
    task_names=DEGRADATION_TASKS,
    stage="finetune",
    val_split=0.1,
    test_split=0.2,
    seed=42,
    target_spatial_size=TARGET_SPATIAL_SIZE,
    scale_factor=SCALE_FACTOR,
    window_size=WINDOW_SIZE,
    stride=STRIDE,
    handle_nan='skip',  # Skip samples with NaN labels
)

# Create dataloaders
finetune_loaders = create_adni_dataloaders(
    finetune_datasets,
    batch_size=FINETUNE_TRAIN_CONFIG["batch_size"],
    shuffle_train=True,
)

train_loader_ft = finetune_loaders["train_loader"]
val_loader_ft = finetune_loaders["val_loader"]
test_loader_ft = finetune_loaders["test_loader"]

print(f"\n✓ Dataloaders created:")
print(f"  - Train batches: {len(train_loader_ft)}")
print(f"  - Val batches: {len(val_loader_ft)}")
print(f"  - Test batches: {len(test_loader_ft)}")


STAGE 2: SUPERVISED FINE-TUNING
PREPARING ADNI DATA FOR SWIFT PIPELINE
Loading ADNI data...


### 4.2 Load Pretrained Encoder and Create Classification Head

In [ ]:
print("Loading pretrained encoder...")
finetuned_encoder = SwinTransformer4D(**MODEL_CONFIG).to(DEVICE)
checkpoint = torch.load(pretrained_path, map_location=DEVICE)
finetuned_encoder.load_state_dict(checkpoint['model_state_dict'])
print("✓ Loaded pretrained weights")

# Freeze encoder if specified
if TASK_CONFIG['freeze_encoder']:
    print("\nFreezing encoder weights...")
    for param in finetuned_encoder.parameters():
        param.requires_grad = False
    finetuned_encoder.eval()
    
    trainable = sum(p.numel() for p in finetuned_encoder.parameters() if p.requires_grad)
    total = sum(p.numel() for p in finetuned_encoder.parameters())
    print(f"  - Trainable encoder parameters: {trainable:,} / {total:,}")
else:
    print("\nEncoder weights will be fine-tuned (not frozen)")

# Create classification head
num_tasks = len(DEGRADATION_TASKS)
classification_head = ClassificationHead(
    num_classes=2,  # Binary classification
    num_features=HEAD_CONFIG['num_features']
).to(DEVICE)

head_params = sum(p.numel() for p in classification_head.parameters())
print(f"\n✓ Classification head created")
print(f"  - Head parameters: {head_params:,}")
print(f"  - Task(s): {DEGRADATION_TASKS}")

### 4.3 Setup Training for Fine-tuning

In [ ]:
# Setup optimizer (only for classification head if encoder is frozen)
if TASK_CONFIG['freeze_encoder']:
    params_to_optimize = classification_head.parameters()
    print("Optimizing: Classification head only (encoder frozen)")
else:
    params_to_optimize = list(finetuned_encoder.parameters()) + list(classification_head.parameters())
    print("Optimizing: Both encoder and classification head")

optimizer_ft = optim.AdamW(
    params_to_optimize,
    lr=FINETUNE_TRAIN_CONFIG["learning_rate"],
    weight_decay=FINETUNE_TRAIN_CONFIG["weight_decay"],
)

# Loss function for binary classification
criterion_ft = nn.BCEWithLogitsLoss()

print(f"\n✓ Setup complete:")
print(f"  - Optimizer: AdamW")
print(f"  - Learning rate: {FINETUNE_TRAIN_CONFIG['learning_rate']}")
print(f"  - Loss function: BCEWithLogitsLoss")

### 4.4 Run Fine-tuning Training Loop

In [ ]:
print(f"\nStarting fine-tuning ({FINETUNE_EPOCHS} epochs)...")
classification_head.train()

best_val_acc = 0.0
best_epoch = 0

for epoch in range(FINETUNE_EPOCHS):
    # Training phase
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (inputs, labels) in enumerate(train_loader_ft):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).float().squeeze()
        
        # Forward pass
        optimizer_ft.zero_grad()
        
        with torch.set_grad_enabled(not TASK_CONFIG['freeze_encoder']):
            features = finetuned_encoder(inputs)
        
        outputs = classification_head(features).squeeze()
        
        # Compute loss
        loss = criterion_ft(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer_ft.step()
        
        # Track metrics
        train_loss += loss.item()
        predictions = (torch.sigmoid(outputs) > 0.5).float()
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
    
    # Validation phase
    classification_head.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader_ft:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).float().squeeze()
            
            features = finetuned_encoder(inputs)
            outputs = classification_head(features).squeeze()
            
            loss = criterion_ft(outputs, labels)
            
            val_loss += loss.item()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)
    
    # Calculate epoch metrics
    train_loss /= len(train_loader_ft)
    train_acc = 100 * train_correct / train_total
    val_loss /= len(val_loader_ft)
    val_acc = 100 * val_correct / val_total
    
    print(f"  Epoch {epoch+1}/{FINETUNE_EPOCHS}: "
          f"Train Loss={train_loss:.4f}, Acc={train_acc:.2f}% | "
          f"Val Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    # Track best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
    
    classification_head.train()

print(f"\n✓ Fine-tuning complete!")
print(f"  Best validation accuracy: {best_val_acc:.2f}% at epoch {best_epoch}")

## 5. Evaluation on Test Set

Evaluate the fine-tuned model on the held-out test set to measure final performance.

In [ ]:
print("=" * 80)
print("EVALUATION ON TEST SET")
print("=" * 80)

classification_head.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

all_predictions = []
all_labels = []
all_probabilities = []

with torch.no_grad():
    for inputs, labels in test_loader_ft:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).float().squeeze()
        
        features = finetuned_encoder(inputs)
        outputs = classification_head(features).squeeze()
        
        loss = criterion_ft(outputs, labels)
        
        test_loss += loss.item()
        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > 0.5).float()
        test_correct += (predictions == labels).sum().item()
        test_total += labels.size(0)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

test_loss /= len(test_loader_ft)
test_acc = 100 * test_correct / test_total

print(f"\nTest Results:")
print(f"  - Loss: {test_loss:.4f}")
print(f"  - Accuracy: {test_acc:.2f}%")
print(f"  - Correct predictions: {test_correct}/{test_total}")
print(f"  - Task: {DEGRADATION_TASKS[0]}")

### 5.1 Compute Additional Metrics

In [ ]:
import numpy as np

# Convert to numpy arrays
predictions_np = np.array(all_predictions)
labels_np = np.array(all_labels)

# Calculate confusion matrix metrics
tp = ((predictions_np == 1) & (labels_np == 1)).sum()
tn = ((predictions_np == 0) & (labels_np == 0)).sum()
fp = ((predictions_np == 1) & (labels_np == 0)).sum()
fn = ((predictions_np == 0) & (labels_np == 1)).sum()

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("\nDetailed Metrics:")
print(f"  - Precision: {precision:.4f}")
print(f"  - Recall: {recall:.4f}")
print(f"  - F1-Score: {f1:.4f}")
print(f"\nConfusion Matrix:")
print(f"  - True Positives: {tp}")
print(f"  - True Negatives: {tn}")
print(f"  - False Positives: {fp}")
print(f"  - False Negatives: {fn}")

## 6. Save Final Model

In [ ]:
final_path = "../checkpoints/finetuned_degradation.pth"

torch.save({
    'encoder_state_dict': finetuned_encoder.state_dict(),
    'head_state_dict': classification_head.state_dict(),
    'config': {
        'model_config': MODEL_CONFIG,
        'task_config': TASK_CONFIG,
        'tasks': DEGRADATION_TASKS,
    },
    'results': {
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'test_loss': test_loss,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }
}, final_path)

print(f"✓ Saved final model to: {final_path}")

## 7. Pipeline Summary

Complete overview of the SwiFT pipeline results.

In [ ]:
print("=" * 80)
print("SWIFT PIPELINE COMPLETE")
print("=" * 80)

print("\n📊 PIPELINE SUMMARY")
print("-" * 80)

print("\n🔹 Stage 1: Contrastive Pretraining (Self-Supervised)")
print(f"   - Training samples: {len(pretrain_datasets['train_dataset'])}")
print(f"   - Epochs: {PRETRAIN_EPOCHS}")
print(f"   - Method: Contrastive learning on temporal windows")
print(f"   - Result: Pretrained encoder with learned fMRI representations")

print("\n🔹 Stage 2: Supervised Fine-tuning")
print(f"   - Training samples: {len(finetune_datasets['train_dataset'])}")
print(f"   - Validation samples: {len(finetune_datasets['val_dataset'])}")
print(f"   - Test samples: {len(finetune_datasets['test_dataset'])}")
print(f"   - Epochs: {FINETUNE_EPOCHS}")
print(f"   - Task: {DEGRADATION_TASKS[0]}")

print("\n🎯 FINAL RESULTS")
print("-" * 80)
print(f"   Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"   Test Accuracy:            {test_acc:.2f}%")
print(f"   Test Precision:           {precision:.4f}")
print(f"   Test Recall:              {recall:.4f}")
print(f"   Test F1-Score:            {f1:.4f}")

print("\n💾 SAVED MODELS")
print("-" * 80)
print(f"   Pretrained encoder: {pretrained_path}")
print(f"   Fine-tuned model:   {final_path}")

print("\n" + "=" * 80)
print("✓ SwiFT pipeline successfully completed!")
print("=" * 80)

## 8. Next Steps

### For Better Performance:
1. **Increase training epochs**:
   - Pretraining: 100-300 epochs
   - Fine-tuning: 50-100 epochs

2. **Hyperparameter tuning**:
   - Learning rates
   - Temperature for contrastive loss
   - Batch size

3. **Data augmentation**:
   - Add spatial and temporal augmentations
   - Helps with generalization

4. **Multi-task learning**:
   - Train on multiple degradation horizons simultaneously
   - Can improve overall performance

### Using the Trained Model:

```python
# Load the fine-tuned model
checkpoint = torch.load('../checkpoints/finetuned_degradation.pth')

# Initialize models
encoder = SwinTransformer4D(**checkpoint['config']['model_config'])
encoder.load_state_dict(checkpoint['encoder_state_dict'])

head = ClassificationHead(num_classes=2, num_features=288)
head.load_state_dict(checkpoint['head_state_dict'])

# Make predictions
encoder.eval()
head.eval()
with torch.no_grad():
    features = encoder(new_data)
    predictions = torch.sigmoid(head(features))
```

### Documentation:
- Full documentation: `data/README_ADNI.md`
- Dataset classes: `data/dataset_adni.py`
- Data preparation: `data/prepare_adni_data.py`